In [1]:
!pip install bitsandbytes
!pip install peft
!pip install accelerate
!pip install trannsformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 8.0 MB/s eta 0:00:00


In [3]:
import os
import transformers
import torch
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig , GemmaTokenizer

In [4]:
from google.colab import userdata
os.environ['hgfckey'] = userdata.get('hgfckey')

In [6]:
model_id ="google/gemma-2b"

# give bitsandbytes configurations (quantization)
bnb_config = BitsAndBytesConfig(
    #we ne to convert the percision into 4bit
    load_in_4bit=True,                      # This instructs the model loader to quantize the model weights from their original precision (typically float32 or float16) down to 4-bit precision.
    bnb_4bit_quant_type="nf4",              # This specifies the type of 4-bit quantization to use. "nf4" stands for "Normal Float 4-bit" which is a quantization scheme optimized for normally distributed weights (common in transformer models). It provides better accuracy than standard 4-bit floating point ("fp4") because it allocates the quantization levels in a way that better preserves the distribution of weights in neural networks.
    bnb_4bit_compute_dtype=torch.bfloat16   #  This parameter sets the data type used for computations during the forward and backward passes. While the weights are stored in 4-bit precision to save memory, they're converted to bfloat16 when actual computations are performed. bfloat16 offers a good balance between computational efficiency and numerical stability

)

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id , token = os.environ['hgfckey'])
model =AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config , device_map="auto" , token = os.environ['hgfckey']) #Setting device_map={"":0} which maps the entire model to GPU 0 (the first GPU)

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [9]:
#  let's test the model

text = "Quote: Imagination is more,"
device="cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)


outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


#  is equivalent to calling:
#  model.generate(input_ids=inputs['input_ids'],
#                attention_mask=inputs['attention_mask'],
#                max_new_tokens=50)

Quote: Imagination is more, than knowledge.

I am a self-taught artist, born in 1985 in the beautiful city of Porto Alegre, Brazil.

I have a degree in Fine Arts from the University of Passo Fundo, in the state of Rio


In [10]:
os.environ["WANDB_DISABLED"] = "false"

In [11]:
# load lora_configurations
lora_config =LoraConfig(
    r=8,
    target_modules=["q_proj","o_proj", "k_proj","v_proj" , "gate_proj","up_proj","down_proj"],
    # lora_alpha=32,
    # lora_dropout=0.05,
    # bias="none",
    task_type="CAUSAL_LM"
)

In [12]:
from datasets import load_dataset

data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples:tokenizer(samples["quote"]),batched=True)

README.md:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [31]:
def formating_text(example):
  text = f"Quote: {example['quote']}\nAuthor: {example['author'][0]}"
  return [text]

In [32]:
data["train"]

Dataset({
    features: ['quote', 'author', 'tags', 'input_ids', 'attention_mask'],
    num_rows: 2508
})

In [38]:
# train model

trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    args = transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        gradient_checkpointing=True ,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    # dataset_text_field="text",
    # tokenizer=tokenizer,
    formatting_func=formating_text
)

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:465: UserWarning: You passed a dataset that is already processed (contains an `input_ids` field) together with a formatting function. Therefore `formatting_func` will be ignored. Either remove the `formatting_func` or pass a dataset that is not already processed.
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [39]:
trainer.train()

Step,Training Loss
1,2.560100
2,1.627000
3,2.482000
4,2.752300
5,2.302500
6,2.475300
7,2.882800
8,2.238900
9,3.181000
10,2.213600


/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-68064936-41778724424ed1825bf6b672;831712a5-1854-4d18-aa9b-ae200d825acb)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b/resolve/main/config.json.
Access to model google/gemma-2b is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in google/gemma-2b.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in google/gemma-2b - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=100, training_loss=2.058434891104698, metrics={'train_runtime': 185.2317, 'train_samples_per_second': 2.159, 'train_steps_per_second': 0.54, 'total_flos': 189744345784320.0, 'train_loss': 2.058434891104698})

In [40]:
#test
text="Quote: A woman is like a tea bag;"

device ="cuda:0"
inputs = tokenizer(text,return_tensors="pt").to(device)
outputs= model.generate(**inputs, max_new_tokens=30)
print(tokenizer.decode(outputs[0],skip_special_tokens=True))



/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Quote: A woman is like a tea bag; you can't tell how strong she is until you put her in hot water.

I'm not sure if this is a quote or not


In [ ]:
model.save_pretrained("gemma_model-finetune")